In [1]:
dimension = 3
perimeter = 50
sensitivity = 1
jsonfilename = "dummyfile.json"

# Generating new examples

This notebook's purpose is to generate new examples of *dysfunctional matrices* - matrices for which the cube algorithm gives different result than the LLL algorithm. For now, this code generates only square matrices. It returns a json file containing the *dysfunctional* matrices and their core parameters.

The generating function takes in parameters above.

`dimension`: the shape of the matrix

`perimeter`: defines an interval `[- perimeter, perimeter]` for the integers inside the matrix

`sensitivity`: lol i forgot

`jsonfilename`: name of the ouput file

## Output format

Jsonfile containing a `list` of `dictionaries`, each dictionary representing single case of a *dysfunctional matrix*. The `dict` includes the following: 

`"B"` the original faulty matrix `B`

`"G"` it's corresponding Gram matrix

`"lincomb_LLL"` linear combination corresponding to the shortest vector according to the LLL algorithm

`"lincomb_cube"` linear combination corresponding to the shortest vector according to the cube algorithm

`"sv_LLL"` shortest vector according to LLL 

`"sv_cube"` shortest vector according to cube

`"lincomb_diff"` difference between the two linear combination (their matrices subtracted)

`"Diff impact"` detailed matrix multiplication of the difference matrix


## Code and commentary

### Utilitary math tools

These functions are uniteresting tools needed for the basic computations. Functions such as *finding the shortest vectors of a basis matrix*, *creating a random linearly independent basis matrix*, *computing Gram matrix of a given matrix* or *converting between data types*

In [2]:
# UNINTERESTING TOOLS
from random import randint, seed


def shortestVector(matrix): 
    """
    :returns: 
        - n - norm of the shortest vector
        - v - the shortest vector (vector)
        - i - index of the SV
    """
    return sorted([(matrix[idx].norm().n(), matrix[idx], idx) for idx in range(matrix.nrows())])[0]

def randomMatrix(dimension, per) -> matrix:
    list = [randint(-per, per) for _ in range(dimension**2)]
    M = matrix(ZZ, dimension, dimension, list)
    while M.rank() != dimension:
        list = [randint(-per, per) for _ in range(dimension**2)]
        M = matrix(ZZ, dimension, dimension, list)
    return M

def gram_matrix(matrix) -> matrix:
    return matrix * matrix.transpose()

def matrix_to_list(A) -> list:
    return [[int(A.list()[row * A.ncols() + col]) for col in range(A.ncols())] for row in range(A.nrows())]

def vector_to_list(vector) -> list:
    return [float(num) for num in vector.list()]

### Interesting math tools. 

Following functions actually have some thoughts behind them. 


`abnormality_data`
Given a matrix $B$ and it's shortest vector given by the LLL algorithm, returns __a dictionary__ with data concerting the SV given by the cube algorithm, if it's far enough from the LLL solution. 

`find_real_minimum` Given a matrix $B$ and it's shortest vector given by the LLL algorithm, returns the __linear combination of the SV given by the cube algorithm__.

`matrix_multiplication_detailed`
Given a matrix $A$ and a vector $v$, computes a detailed matrix multiplication $v \cdot A \cdot v^T$ in a sense, that every cell in the matrix is multiplicated with the corresponding cell of the vector. The goal is to have clearer perspective on which cells in the matrix are effecting the multiplication the most. 
Return this detailed matrix and a sum of it's elements, resp. the result of $v \cdot A \cdot v^T$.




In [3]:
### batshit cool math
def abnormality_data(B, v_min, lcLLL, G, current_row) -> dict:
    lcCube = find_real_minimum(G, current_row, lcLLL[current_row])
    if lcCube == zero_vector(SR, dimension): return
    for i in range(dimension - 1):  
        difference = lcLLL[i] - lcCube[i]
        if abs(difference) > sensitivity:
            return into_dict(B, lcLLL, lcCube)
        
def find_real_minimum(G, current_row, lcLLL) -> vector:
    matrixA = matrix(dimension - 1, dimension - 1, 0)
    matrixB = matrix(dimension - 1, 1, 0)
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([lcLLL * G[row,j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right((-1) * matrixB)).list()
    result.insert(current_row, lcLLL)
    return vector(result).n(digits=5) 


def matrix_multiplication_detailed(A, vector) -> (matrix, float):
    """
    :returns: 
        - A - matrix A multiplied in rows and cols by vector
        - x - sum of all elements in the returned matrix
    """
    A = matrix_to_list(A)
    n = len(A)
    vector = vector_to_list(vector)
    for i in range(n):
        for j in range(n):
            A[i][j] = round(A[i][j]*  vector[i] * vector[j], 2)
    return matrix(A), round(sum(sum(A, [])), 2)

### Formatting tools

These functions provide transfering the data to the output jsonfile as described above.

In [4]:
### output formatting
import json

def format_data(output_data):
    for dic in output_data:
        for key, value in dic.items():
            if isinstance(value, sage.matrix.matrix_integer_dense.Matrix_integer_dense):
                dic[key] = matrix_to_list(value)
            else:
            # FIXME here I assume that output_data consits only of matrices and one-line entries (vectors, free modules)
            # it IS going to crash if I try to save integers/floats
                dic[key] = vector_to_list(value)
    return output_data

def into_json(output_data):
    out_file = open(jsonfilename, "w+") 
    json.dump(output_data, out_file) 
    out_file.close() 

def from_json(filename):
    out_file = open(filename)
    return json.load(out_file)

def print_listdict(list) -> None:
    """
    :param list: list of dictionaries
    """
    for dictionary in list:
        for pair in dictionary.items():
            print(pair[1], ": ", pair[0])
        print()
    
def into_dict(B, lcLLL: vector, lcCube: vector) -> dict:
    result = {}
    G = gram_matrix(B)
    result["B"] = B
    result["G"] = G
    result["lincomb_LLL"] = lcLLL
    result["lincomb_cube"] = lcCube
    result["sv_LLL"] = lcLLL * B
    result["sv_cube"] = lcCube * B
    result["lincomb_diff"] = lcLLL - vector(lcCube)
    result["Diff impact"] = matrix_multiplication_detailed(G, lcLLL - lcCube)[0]
#     result["Diff impact sum"] = matrix_multiplication_detailed(B, lcLLL - lcCube)[1]
#     result["LLL.norm"] = vector(lcLLL*B).norm().n(digits=5)
#     result["cube.norm"] = (vector(lcCube)*B).norm().n(digits=5)
    return result
    

### Main function

This cell actually generates the new examples and checks wheter a matrix is *dysfunctional*. 

`generate_new_examples`
The main function. Based on input parameters above, generates *some* number of *dysfunctional* matrices, computes their invariants such as Gram matrix, LLL/cube linear combinations etc. and creates a json file with this information (specified above).

`abnormality_check`
Given a matrix $B$, checks whether....?


In [32]:
# MAIN 
def generate_new_examples(iterations, printing = False) -> None:
    output_data = []
    for _ in range(iterations):
        B = randomMatrix(dimension, perimeter)
        if abnormality_check(B): output_data.append(abnormality_check(B))
    # output_data = list(filter(lambda item: item is not None, output_data)) # delete nones
    # this is prolly not neccessary tx to return False and if above
    print(output_data)
    if printing: print_listdict(output_data)
    into_json(format_data(output_data))


def abnormality_check(B) -> dict:
    v_min = shortestVector(B.LLL())[1]
    lcLLL = B.solve_left(v_min)
    G = gram_matrix(B)
    for current_row in range(dimension):
        x = abnormality_data(B, v_min, lcLLL, G, current_row)
        if x is not None: return x
    return False
# These nones are different from thow on line 8. These come up when a line is ok. Those on line 8 come up when the whole matrix is ok
 

In [34]:
# generate_new_examples(500, True)